In [1]:
import pyam
import pandas as pd

from pathlib import Path

<IPython.core.display.Javascript object>

Step 1: Read in the necessary data

In [2]:
df = pyam.IamDataFrame(
    Path(
        '../data/101_data.xlsx'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/101_data.xlsx
pyam.core - INFO: Reading meta indicators


Step 2: Filter out for the necessary variables that we will use.

In [3]:
df.variable

['AR6 Reanalysis|OSCARv3.2|Carbon Removal',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only',
 'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only',
 'Population']

In [4]:
df_vars = (
    df
    .filter(
        region='World',
        variable=[
            'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only',
            'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
            'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'
        ]
    )
)

In [5]:
df_vars.convert_unit(
    current='Gt CO2-equiv/yr',
    to='Gt CO2/yr',
    factor=1,
    inplace=True
)

Step 3: Aggregate the gross emissions

In [6]:
df_vars.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Emissions|Gross',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
        'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only'
    ],
    append=True
)

Step 4: Split the dataframe into two parts:
* The net GHG emissions that we will use to estimate the total mitigation effort
* The two component mitigation options we will use for proportions.

In [7]:
df_total_mitigation = (
    df_vars
    .filter(variable='AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only')
    .timeseries()
)

In [8]:
df_components_mitigation = (
    df_vars
    .filter(
        variable=[
            'AR6 Reanalysis|OSCARv3.2|Emissions|Gross',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'
        ]
    )
    .offset(year=2020)
    .timeseries()
)

Step 5: Pull out the net zero CO2 years

In [9]:
netzero_years = df_vars.meta.loc[:, 'Year of netzero CO2 emissions (Harm-Infilled) table']

In [10]:
netzero_years.fillna(2100, inplace=True)

Step 6: First, let us get the 2020 and net zero CO2 values of the net GHG emissions.

In [11]:
net_ghg_2020 = (
    pd.DataFrame(df_total_mitigation[2020])
    .droplevel(['region', 'variable', 'unit'])
    .round(0)
)

In [12]:
net_ghg_netzero = (
    pd.DataFrame(
        df_total_mitigation.apply(
            lambda x: x[netzero_years.loc[x.name[0:2]]],
            axis=1
        )
    )
    .droplevel(['region', 'variable', 'unit'])
    .round(0)
    .rename(
        columns={0:'netzero'}
    )
)

In [13]:
net_ghg_compiled = pd.concat([net_ghg_2020, net_ghg_netzero], axis=1)

In [14]:
net_ghg_compiled.loc[:,'Category'] = df_vars.meta.loc[:, 'Category']

Step 7: Compile a dataframe with the difference between 2020 and net zero for the two mitigation component variables.

In [15]:
components_compiled = (
    df_components_mitigation.apply(
        lambda x: x[netzero_years.loc[x.name[0:2]]],
        axis=1
    )
    .droplevel(['region', 'unit'])
    .unstack(level=-1)
    .rename(
        columns={
            'AR6 Reanalysis|OSCARv3.2|Emissions|Gross':'reduce',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total':'remove'
        }
    )
)

In [16]:
components_compiled.loc[:, 'remove'] *= -1

In [17]:
components_compiled

variable                                      remove     reduce
model             scenario                                     
AIM/CGE 2.0       SSP1-26                  -7.675157 -39.924308
                  SSP4-26                  -7.395017 -43.153110
AIM/CGE 2.1       CD-LINKS_NPi2020_1000   -12.240858 -35.431404
                  CD-LINKS_NPi2020_400    -16.143932 -40.820973
                  CO_Bridge                -7.373163 -40.490306
...                                              ...        ...
WITCH-GLOBIOM 4.4 CD-LINKS_INDC2030i_1600 -12.900026 -36.657457
                  CD-LINKS_NDC2030i_1000   -9.897303 -40.806546
                  CD-LINKS_NPi2020_1000    -9.834387 -42.161808
                  CD-LINKS_NPi2020_1600   -10.857048 -39.866954
                  CD-LINKS_NPi2020_400     -8.520957 -44.066332

[407 rows x 2 columns]

Step 8: Create a composite dataframe that we will use to plot stuff. Write it out for plotting.

In [18]:
figure2_panelc = pd.concat(
    [net_ghg_compiled, components_compiled],
    axis=1
)

In [19]:
figure2_panelc.to_csv(
    Path(
        '../data/104_figure2_panelc.csv'
    )
)

Step 9: Calculate the proportion of net GHG reductions met via reductions.

In [20]:
figure2_panelc.loc[:, 'total_reduction'] = (
    figure2_panelc.loc[:, 'netzero']
    -
    figure2_panelc.loc[:, 2020]
)

In [21]:
figure2_panelc.loc[:, 'prop_reduce'] = (
    figure2_panelc.loc[:, 'reduce'] * 100
    /
    figure2_panelc.loc[:, 'total_reduction']
).round(0)

In [22]:
figure2_panelc.groupby('Category')['prop_reduce'].describe()

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
C1,70.0,84.142857,6.649149,72.0,79.25,85.0,88.0,102.0
C2,106.0,81.830189,6.901307,53.0,77.00,82.5,87.0,93.0
C3,231.0,81.532468,7.312737,52.0,75.00,83.0,87.0,101.0
